In [157]:
import pandas as pd
import regex as re
import numpy as np


idGenerator = 0
teamIDGenerator = 0

def getClubId():
    global idGenerator
    res = 'CLB' + '%03d' % idGenerator
    idGenerator +=1
    return res


def getTeamId():
    global teamIDGenerator
    res = 'T' + '%05d' % teamIDGenerator
    teamIDGenerator +=1
    return res

In [158]:
fileSaveDir = r'C:\Users\Chaconne\Documents\学业\UMD\Assignments\703_DataBase\GroupProject\InUseData\SavedCSV.csv'

tab = pd.read_csv(fileSaveDir)

tab.reset_index(inplace=True, drop=True)

In [159]:
tab['Tri-code'] = tab['Tri-code'].ffill()

tab

,League,Team,Official Summoner Name,Main Role,Legal First Name,Legal Family Name,"End Date (Month, Day, Year)",Tri-code
0,LPL,FunPlus Phoenix,Lwx,ADC,WEIXIANG,LIN,22-Nov-22,FPX
1,LPL,FunPlus Phoenix,jiang,ADC,BAIJIANG,LUO,22-Nov-22,FPX
2,LPL,FunPlus Phoenix,xiaolaohu,TOP,XIAOHU,PING,22-Nov-22,FPX
3,LPL,FunPlus Phoenix,Beichuan,JUG,LING,YANG,21-Nov-23,FPX
4,LPL,FunPlus Phoenix,Hang,SUP,MINGHANG,FU,21-Nov-23,FPX
...,...,...,...,...,...,...,...,...
330,LDL,MAX,KennyChan,JUG,CHIYUNG,CHAN,22-Nov-22,MAX
331,LDL,MAX,Xiaguang,ADC,LINGFENG,WU,22-Nov-22,MAX
332,LDL,MAX,XinMo,SUP,QIANXI,LI,22-Nov-22,MAX
333,LDL,MAX,Medzz,MID,KAYUEN,LI,21-Nov-23,MAX


In [160]:
clbName = tab.Team.astype(str)
clbTriCode = tab['Tri-code']
clbRegion = tab.League.astype(str)

clubTab = pd.DataFrame({'clbName': clbName, 'clbRegion': clbRegion, 'Tri-code':clbTriCode})
clubTab.drop_duplicates(inplace=True)
clubTab.dropna(inplace= True, axis=0, how='all')


clubTab.set_index(inplace= True, drop=True, keys=['Tri-code'])

clubTab

,clbName,clbRegion
Tri-code,,
FPX,FunPlus Phoenix,LPL
FPX,nan,nan
IG,InvictusGaming,LPL
IG,nan,nan
BLG,BLGPingAnBank,LPL
...,...,...
TP,TeamPinnacle,LDL
TP,nan,nan
Q9,Qing Jiu E-sport Club,LDL


In [161]:
duplicateClubs = ['JDG', 'FPX', 'RNG', 'OMG']


clubTab.drop(axis= 0, inplace= True, labels= duplicateClubs)

In [162]:
clubTab = clubTab.iloc[::2, :]

clubTab['clbId'] = clubTab.clbName.map(lambda s : getClubId())

clubTab = clubTab[['clbId', 'clbName',	'clbRegion']]

clubTab.to_csv('ClubCSV.csv')

C:\Users\Chaconne\AppData\Local\Temp\ipykernel_33304\3349938027.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clubTab['clbId'] = clubTab.clbName.map(lambda s : getClubId())


In [163]:
teamTab = pd.DataFrame({'temName': clbName, 'Tri-code':clbTriCode, 'clbRegion': clbRegion}).drop_duplicates().dropna()

teamTab = teamTab.iloc[::2, :]

teamTab.reset_index(drop=True, inplace=True)

teamAndClubDf = pd.merge(teamTab, clubTab, on='Tri-code', how='inner')

teamAndClubDf = teamAndClubDf[teamAndClubDf['clbRegion_x'] == 'LPL']

In [164]:
finalTeamTab = pd.DataFrame({'temName':teamAndClubDf.temName, 'temTriCode':teamAndClubDf['Tri-code'], 'clbId':teamAndClubDf.clbId})
finalTeamTab['temId'] = finalTeamTab.index.map(lambda s : getTeamId())

finalTeamTab

,temName,temTriCode,clbId,temId
0,InvictusGaming,IG,CLB000,T00000
1,BLGPingAnBank,BLG,CLB001,T00001
2,THUNDERTALKGAMING,TT,CLB002,T00002
3,EDWARDGAMING,EDG,CLB003,T00003
4,SuzhouLNGEsports,LNG,CLB004,T00004
5,ShenzhenVictoryFive,V5,CLB005,T00005
6,AnyonesLegend,AL,CLB006,T00006
7,WeiboGaming,WBG,CLB007,T00007
8,TOPESPORTS,TES,CLB008,T00008
9,RareAtom,RA,CLB009,T00009


In [165]:
finalTeamTab.to_csv('TeamCSV.csv')